In [ ]:
!pip install pandas numpy scikit-rf

import pandas as pd
import numpy as np
import skrf as rf

# Load CSV data
file_path = 'S Parameter Plot 4.csv'
df = pd.read_csv(file_path)

# Extract frequency and convert from GHz to Hz
freq_ghz = df['Freq [GHz]'].values
freq_hz = freq_ghz * 1e9

# Extract S-parameters and form complex values
s11 = df['re(S(1,1)) []'].values + 1j * df['im(S(1,1)) []'].values
s12 = df['re(S(1,2)) []'].values + 1j * df['im(S(1,2)) []'].values
s21 = df['re(S(2,1)) []'].values + 1j * df['im(S(2,1)) []'].values
s22 = df['re(S(2,2)) []'].values + 1j * df['im(S(2,2)) []'].values

# Create S-parameter matrix of shape (n_freqs, 2, 2)
s_params = np.array([[s11, s12], [s21, s22]])  # shape (2, 2, N)
s_params = np.transpose(s_params, (2, 0, 1))   # reshape to (N, 2, 2)

# Create frequency object and network
frequency = rf.Frequency.from_f(freq_hz, unit='Hz')
network = rf.Network(frequency=frequency, s=s_params, name='S_to_ABCD')

# Convert to ABCD parameters
abcd_params = network.a  # Returns ABCD matrix at each frequency

# Optional: Print ABCD parameters for each frequency
for f, abcd in zip(freq_hz, abcd_params):
    print(f"Frequency: {f/1e9:.2f} GHz")
    print("ABCD Matrix:")
    print(abcd)
    print("-" * 40)

abcd_list = []
for f, abcd in zip(freq_ghz, abcd_params):
    abcd_list.append({
        'Freq (GHz)': f,
        'A_real': abcd[0, 0].real, 'A_imag': abcd[0, 0].imag,
        'B_real': abcd[0, 1].real, 'B_imag': abcd[0, 1].imag,
        'C_real': abcd[1, 0].real, 'C_imag': abcd[1, 0].imag,
        'D_real': abcd[1, 1].real, 'D_imag': abcd[1, 1].imag,
    })

# Create and save DataFrame
abcd_df = pd.DataFrame(abcd_list)
abcd_df.to_csv('ABCD_Parameters.csv', index=False)


Frequency: 5.00 GHz
ABCD Matrix:
[[1.03424654e+02+8.92345686e-02j 5.16878191e+03+4.45968839e+00j]
 [2.06926176e+00+1.78567421e-03j 1.03423723e+02+8.92345449e-02j]]
----------------------------------------
Frequency: 5.10 GHz
ABCD Matrix:
[[8.88372494e+01+7.93660209e-02j 4.43970503e+03+3.96659343e+00j]
 [1.77736697e+00+1.58820756e-03j 8.88364413e+01+7.93661490e-02j]]
----------------------------------------
Frequency: 5.20 GHz
ABCD Matrix:
[[7.57185618e+01+7.02300395e-02j 3.78401391e+03+3.51010945e+00j]
 [1.51485856e+00+1.40540762e-03j 7.57178392e+01+7.02302632e-02j]]
----------------------------------------
Frequency: 5.30 GHz
ABCD Matrix:
[[6.39903182e+01+6.18039539e-02j 3.19779574e+03+3.08910916e+00j]
 [1.28016879e+00+1.23682215e-03j 6.39896536e+01+6.18042264e-02j]]
----------------------------------------
Frequency: 5.40 GHz
ABCD Matrix:
[[5.35713847e+01+5.40636162e-02j 2.67699352e+03+2.70239332e+00j]
 [1.07167229e+00+1.08197008e-03j 5.35707595e+01+5.40638990e-02j]]
----------------